# **Inatall Libraries**

In [ ]:
!pip install apify_client langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=72b674da9b9f632e0c934bc941184119bed85de6046d0c479ebfc0b43d79fa76
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


# **Run Basic API**

In [ ]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_XXXX")

# Prepare the Actor input
run_input = {
    "startUrls": [
        "https://www.aliexpress.com/item/1005004189740905.html",
    ],
    "maxItems": 10,
    "searchInSubcategories": True,
    "language": "en_US",
    "shipTo": "US",
    "currency": "USD",
    "includeDescription": False,
    "maxFeedbacks": 40,
    "maxQuestions": 5,
    "proxy": { "useApifyProxy": True },
    "extendOutputFunction": "($) => { return {} }",
}

# Run the Actor and wait for it to finish
run = client.actor("xYHAh66JcxDGnzs4G").call(run_input=run_input)

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

{'id': 3256804003426153, 'link': 'https://www.aliexpress.com/item/1005004189740905.html', 'ownerMemberId': 240612995, 'title': 'Ordenador portátil con pantalla IPS de 15,6 pulgadas, 16GB de RAM, Intel 11th N5095, Windows 10, 11 Pro, para oficina y juegos', 'averageStar': '4.8', 'soldCount': None, 'descriptionURL': 'https://aeproductsourcesite.alicdn.com/product/description/pc/v2/es_ES/desc.htm?productId=1005004189740905&key=S7f1e7513e56844ce916bea0d7bb178c4y.zip&token=e68d4ae776789db6801c52cc5daeb130', 'store': {'positiveNum': 76, 'positiveRate': '95.8', 'name': 'Professional Laptop Tablet Store', 'id': 910811026, 'url': 'https://www.aliexpress.com/store/1101551533', 'topRatedSeller': False}, 'specs': [{'Producto químico muy preocupado': 'Ninguno'}, {'Uso recomendado': 'Aprendizaje,USO COMERCIAL'}, {'Teclados': 'Teclado retroiluminado,Teclado numérico'}, {'Capacidad de la batería (mAh)': '5000mAH'}, {'Red comunicación': 'Compatible con Bluetooth,Wi-Fi'}, {'Pantalla táctil': 'no'}, {'Al

In [ ]:
import json

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(json.dumps(item, indent=4))  # Pretty-print the JSON


{
    "id": 3256808332256511,
    "link": "https://www.aliexpress.com/item/3256808332256511.html",
    "ownerMemberId": 2671802027,
    "title": "FIREBAT T6A 16 Inch AMD Ryzen7 7435HS Geforce RTX 3050 DDR5 165Hz Wifi6 BT5.1 16GB 512GB Gaming Gamer Notebook Laptop",
    "soldCount": null,
    "descriptionURL": "https://aeproductsourcesite.alicdn.com/product/description/pc/v2/en_US/desc.htm?productId=1005008518571263&key=S5be28b6ff81c4508ada6881d85be3e34J.zip&token=e2f493cea54d201153e4d58a17b6afb2",
    "store": {
        "positiveNum": 104,
        "positiveRate": "89.2",
        "name": "Firebat Official Store",
        "id": 1102531135,
        "url": "https://www.aliexpress.com/store/1102527259",
        "topRatedSeller": false
    },
    "specs": [
        {
            "Hign-concerned Chemical": "None"
        },
        {
            "Recommended use": "Learing,Business Use,Gaming,DESIGN,Daily Entertainment"
        },
        {
            "Keyboards": "TOUCHPAD,Backlit Keyboard,

# **Scrape Review from multiple product with URL and number of reviews**

Define Scraping Functions

In [ ]:
import csv
from apify_client import ApifyClient

# Function to scrape feedback for a given URL and max feedback
def scrape_feedback(client, url, max_feedbacks):
    run_input = {
        "startUrls": [url],
        "maxItems": 1,  # Only one item at a time
        "searchInSubcategories": False,
        "language": "en_US",
        "shipTo": "US",
        "currency": "USD",
        "includeDescription": False,
        "maxFeedbacks": max_feedbacks,
        "maxQuestions": 0,
        "proxy": {"useApifyProxy": True},
        "extendOutputFunction": "($) => { return {} }",
    }

    # Run the Actor and wait for it to finish
    run = client.actor("xYHAh66JcxDGnzs4G").call(run_input=run_input)

    # Collect all feedbacks
    all_feedbacks = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        feedbacks = item.get("userFeedbacks", [])
        for feedback in feedbacks:
            all_feedbacks.append(
                {
                    "productURL": url,
                    "userName": feedback.get("userName", ""),
                    "userCountry": feedback.get("userCountry", ""),
                    "userStar": feedback.get("userStar", ""),
                    "reviewContent": feedback.get("reviewContent", ""),
                    "reviewTime": feedback.get("reviewTime", ""),
                }
            )
    return all_feedbacks


# 🎯 Main function to handle multiple URLs and count total feedbacks
def scrape_multiple_urls(urls, max_feedbacks_list):
    # Initialize the ApifyClient
    #client = ApifyClient("apify_api_nCk6PLhKkqsWubjCtWWZ9CQjzbMw8I0CwLeW")
    client = ApifyClient("apify_api_XXXX")  # Replace with your actual API token

    # Prepare the CSV file
    csv_file = "Final_all_user_feedbacks221.csv"
    total_feedback_count = 0  # Counter for total feedbacks

    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(
            file, fieldnames=["productURL", "userName", "userCountry", "userStar", "reviewContent", "reviewTime"]
        )
        writer.writeheader()  # Write CSV header

        # Iterate over URLs and max feedbacks
        for idx, (url, max_feedbacks) in enumerate(zip(urls, max_feedbacks_list), start=1):
            feedbacks = scrape_feedback(client, url, int(max_feedbacks))
            feedback_count = len(feedbacks)
            total_feedback_count += feedback_count  # Count feedbacks

            # ✅ Print tracking info
            print(f"🔗 URL {idx}/{len(urls)}: {url}")
            print(f"📝 Number of feedbacks collected from this URL: {feedback_count}\n")

            for feedback in feedbacks:
                writer.writerow(feedback)

    print(f"\n✅ All user feedbacks have been saved to {csv_file}")
    print(f"📊 Total number of feedbacks collected: {total_feedback_count}")


1st phase Scraping

In [ ]:

urls = [
"https://www.aliexpress.com/item/1005006876657863.html?spm=a2g0o.productlist.main.29.4b692d3c6MUfau&algo_pvid=567885e4-a76a-4d6a-84d6-b4830f6e63f1&algo_exp_id=567885e4-a76a-4d6a-84d6-b4830f6e63f1-14&pdp_ext_f=%7B%22order%22%3A%224997%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%214913.66%213292.07%21%21%21291.47%21195.28%21%402141122217404120397816133ee933%2112000038593548803%21sea%21BD%210%21ABX&curPageLogUid=VYRRl95NIhMF&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006513094908.html?spm=a2g0o.productlist.main.9.60111e92KYhopW&algo_pvid=0117ce36-3fd2-45ec-8439-aba6cf381615&algo_exp_id=0117ce36-3fd2-45ec-8439-aba6cf381615-4&pdp_ext_f=%7B%22order%22%3A%221327%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211215.44%21607.72%21%21%2110.00%215.00%21%402141001d17404149920752797ef4d0%2112000037484312460%21sea%21BD%210%21ABX&curPageLogUid=I8fzcWrcW9n4&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005008114353969.html?spm=a2g0o.productlist.main.67.60111e92KYhopW&algo_pvid=0117ce36-3fd2-45ec-8439-aba6cf381615&algo_exp_id=0117ce36-3fd2-45ec-8439-aba6cf381615-33&pdp_ext_f=%7B%22order%22%3A%221411%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%213877.39%213644.75%21%21%21230.00%21216.20%21%402141001d17404149920752797ef4d0%2112000043841347904%21sea%21BD%210%21ABX&curPageLogUid=5Iz5Gh0ICx2w&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007174024115.html?spm=a2g0o.productlist.main.1.49187hDT7hDTra&algo_pvid=946755b0-b57c-4a11-be1d-19c296648103&algo_exp_id=946755b0-b57c-4a11-be1d-19c296648103-0&pdp_ext_f=%7B%22order%22%3A%228252%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%213708.80%211891.49%21%21%21220.00%21112.20%21%402141122217404170894933992ee963%2112000039703995407%21sea%21BD%210%21ABX&curPageLogUid=sbMp6RI9FA1z&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007379078015.html?spm=a2g0o.productlist.main.3.49187hDT7hDTra&algo_pvid=946755b0-b57c-4a11-be1d-19c296648103&algo_exp_id=946755b0-b57c-4a11-be1d-19c296648103-1&pdp_ext_f=%7B%22order%22%3A%223092%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211736.39%21868.19%21%21%21103.00%2151.50%21%402141122217404170894933992ee963%2112000040560274372%21sea%21BD%210%21ABX&curPageLogUid=KGJoZOwXc6Vu&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005001281052332.html?spm=a2g0o.productlist.main.33.49187hDT7hDTra&algo_pvid=946755b0-b57c-4a11-be1d-19c296648103&algo_exp_id=946755b0-b57c-4a11-be1d-19c296648103-16&pdp_ext_f=%7B%22order%22%3A%221826%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21685.51%21311.15%21%21%215.64%212.56%21%402141122217404170894933992ee963%2112000029321744038%21sea%21BD%210%21ABX&curPageLogUid=uz9rNNb73Plj&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006581957584.html?spm=a2g0o.productlist.main.43.49187hDT7hDTra&algo_pvid=946755b0-b57c-4a11-be1d-19c296648103&algo_exp_id=946755b0-b57c-4a11-be1d-19c296648103-21&pdp_ext_f=%7B%22order%22%3A%221594%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212714.17%212714.17%21%21%21161.00%21161.00%21%402141122217404170894933992ee963%2112000037725136976%21sea%21BD%210%21ABX&curPageLogUid=BKYTTs939gX3&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006955641146.html?spm=a2g0o.productlist.main.113.49187hDT7hDTra&algo_pvid=946755b0-b57c-4a11-be1d-19c296648103&algo_exp_id=946755b0-b57c-4a11-be1d-19c296648103-56&pdp_ext_f=%7B%22order%22%3A%221441%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21729.26%21729.26%21%21%216.00%216.00%21%402141122217404170894933992ee963%2112000038846547009%21sea%21BD%210%21ABX&curPageLogUid=JcZOR9z0Mcai&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005002628213482.html?spm=a2g0o.productlist.main.19.49187hDT7hDTra&algo_pvid=a3e80203-a4f6-466a-b5fd-15c3153585aa&algo_exp_id=a3e80203-a4f6-466a-b5fd-15c3153585aa-9&pdp_ext_f=%7B%22order%22%3A%222273%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211514.44%21984.51%21%21%2112.46%218.10%21%402140c1c317404186691583205ed51f%2112000028776817730%21sea%21BD%210%21ABX&curPageLogUid=DHjs3UoTnWf5&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005640577095.html?spm=a2g0o.productlist.main.1.2c2f568f8F9ayL&algo_pvid=7137f8ef-3f5e-49fe-b747-a39366af2e9c&algo_exp_id=7137f8ef-3f5e-49fe-b747-a39366af2e9c-0&pdp_ext_f=%7B%22order%22%3A%227545%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21287.43%21135.03%21%21%2117.05%218.01%21%40210156fc17404191327872611eb081%2112000033848748683%21sea%21BD%210%21ABX&curPageLogUid=EqHaTudez2F6&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007559042178.html?spm=a2g0o.productlist.main.9.2c2f568f8F9ayL&algo_pvid=7137f8ef-3f5e-49fe-b747-a39366af2e9c&algo_exp_id=7137f8ef-3f5e-49fe-b747-a39366af2e9c-4&pdp_ext_f=%7B%22order%22%3A%224715%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21599.48%21599.48%21%21%2135.56%2135.56%21%40210156fc17404191327872611eb081%2112000041290047160%21sea%21BD%210%21ABX&curPageLogUid=QKbEkzBOkBtx&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006046852885.html?spm=a2g0o.productlist.main.5.2e3359aeOXHXk4&algo_pvid=9f316319-66d0-4e07-b675-f2c1fd4d4d92&algo_exp_id=9f316319-66d0-4e07-b675-f2c1fd4d4d92-2&pdp_ext_f=%7B%22order%22%3A%221428%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21152.80%2184.05%21%21%219.09%215.00%21%40%2112000035476043397%21sea%21BD%210%21ABX&curPageLogUid=rdosQQD0q6Q5&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005981132478.html?spm=a2g0o.productlist.main.59.2e3359aeOXHXk4&algo_pvid=9f316319-66d0-4e07-b675-f2c1fd4d4d92&algo_exp_id=9f316319-66d0-4e07-b675-f2c1fd4d4d92-29&pdp_ext_f=%7B%22order%22%3A%225188%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21453.34%21294.66%21%21%2126.97%2117.53%21%40%2112000035160207992%21sea%21BD%210%21ABX&curPageLogUid=awmsakII8GY9&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005863377803.html?spm=a2g0o.productlist.main.1.17ffrHZgrHZgeB&algo_pvid=f4426d74-c0b4-472b-9998-b0c8ce256a42&algo_exp_id=f4426d74-c0b4-472b-9998-b0c8ce256a42-0&pdp_ext_f=%7B%22order%22%3A%223504%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21592.86%21255.00%21%21%2135.27%2115.17%21%402101590d17404567922686069e8c0e%2112000034616887109%21sea%21BD%210%21ABX&curPageLogUid=SaZg6WIFNnyW&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005288681440.html?spm=a2g0o.productlist.main.13.17ffrHZgrHZgeB&algo_pvid=f4426d74-c0b4-472b-9998-b0c8ce256a42&algo_exp_id=f4426d74-c0b4-472b-9998-b0c8ce256a42-6&pdp_ext_f=%7B%22order%22%3A%222030%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21258.69%21129.26%21%21%2115.39%217.69%21%402101590d17404567922686069e8c0e%2112000032502243516%21sea%21BD%210%21ABX&curPageLogUid=C6JdL7ZAQHJ5&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006856675635.html?spm=a2g0o.productlist.main.41.17ffrHZgrHZgeB&algo_pvid=f4426d74-c0b4-472b-9998-b0c8ce256a42&algo_exp_id=f4426d74-c0b4-472b-9998-b0c8ce256a42-20&pdp_ext_f=%7B%22order%22%3A%221397%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21269.43%21174.76%21%21%212.22%211.44%21%402101590d17404567922686069e8c0e%2112000043611654636%21sea%21BD%210%21ABX&curPageLogUid=eQHqbJCRs9eY&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006970947603.html?spm=a2g0o.productlist.main.5.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-2&pdp_ext_f=%7B%22order%22%3A%223880%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21181.54%21145.23%21%21%2110.80%218.64%21%40%2112000039144810689%21sea%21BD%210%21ABX&curPageLogUid=btI0zYFoO3NE&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006974148871.html?spm=a2g0o.productlist.main.25.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-12&pdp_ext_f=%7B%22order%22%3A%227204%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21390.65%21234.32%21%21%2123.24%2113.94%21%40%2112000038906193820%21sea%21BD%210%21ABX&curPageLogUid=pR86nLhNHqBX&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006685087877.html?spm=a2g0o.productlist.main.37.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-18&pdp_ext_f=%7B%22order%22%3A%222036%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21537.89%21376.52%21%21%2132.00%2122.40%21%40%2112000038009209044%21sea%21BD%210%21ABX&curPageLogUid=zVRmjTTP5grh&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005008068331179.html?spm=a2g0o.productlist.main.41.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-20&pdp_ext_f=%7B%22order%22%3A%228031%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21188.26%21150.61%21%21%2111.20%218.96%21%40%2112000043524431050%21sea%21BD%210%21ABX&curPageLogUid=txYJjK9Pm7sE&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006662355503.html?spm=a2g0o.productlist.main.43.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-21&pdp_ext_f=%7B%22order%22%3A%222310%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21188.26%21150.61%21%21%2111.20%218.96%21%40%2112000037956346055%21sea%21BD%210%21ABX&curPageLogUid=oOB71rf8xH0y&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005008231723559.html?spm=a2g0o.productlist.main.65.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-32&pdp_ext_f=%7B%22order%22%3A%224823%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21390.65%21234.32%21%21%2123.24%2113.94%21%4021015b2417404571495114900e40da%2112000044314748100%21sea%21BD%210%21ABX&curPageLogUid=2qmOLahMmPZT&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007176551040.html?spm=a2g0o.productlist.main.75.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-37&pdp_ext_f=%7B%22order%22%3A%228314%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21390.65%21234.32%21%21%2123.24%2113.94%21%4021015b2417404571495114900e40da%2112000039706504773%21sea%21BD%210%21ABX&curPageLogUid=SXDdwLakoa0b&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006005046502.html?spm=a2g0o.productlist.main.85.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-42&pdp_ext_f=%7B%22order%22%3A%2217768%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21239.53%2174.30%21%21%2114.25%214.42%21%4021015b2417404571495114900e40da%2112000035282492984%21sea%21BD%210%21ABX&curPageLogUid=6WhtYlqDImNi&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005352839598.html?spm=a2g0o.productlist.main.99.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-49&pdp_ext_f=%7B%22order%22%3A%223632%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21106.91%2172.62%21%21%216.36%214.32%21%4021015b2417404571495114900e40da%2112000032716834509%21sea%21BD%210%21ABX&curPageLogUid=3swLpmjBmS2Z&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004113525812.html?spm=a2g0o.productlist.main.107.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-53&pdp_ext_f=%7B%22order%22%3A%223401%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21148.07%21133.51%21%21%211.22%211.10%21%4021015b2417404571495114900e40da%2112000028076009558%21sea%21BD%210%21ABX&curPageLogUid=eEdHe98YxTEc&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005359350734.html?spm=a2g0o.productlist.main.109.48c15073v6hR7d&algo_pvid=bb42ebf9-6809-4fb2-85cf-dc2a36796d42&algo_exp_id=bb42ebf9-6809-4fb2-85cf-dc2a36796d42-54&pdp_ext_f=%7B%22order%22%3A%222672%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21172.13%21137.67%21%21%2110.24%218.19%21%4021015b2417404571495114900e40da%2112000032735189489%21sea%21BD%210%21ABX&curPageLogUid=4BRl95VdwRYf&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006461510213.html?spm=a2g0o.productlist.main.5.69a766bfTV5HDi&algo_pvid=bfaa59b8-ce29-490e-9f21-f8b2c866d5d7&algo_exp_id=bfaa59b8-ce29-490e-9f21-f8b2c866d5d7-2&pdp_ext_f=%7B%22order%22%3A%224765%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211978.78%211207.07%21%21%21117.72%2171.81%21%402101062a17404578228845116e495c%2112000038354048226%21sea%21BD%210%21ABX&curPageLogUid=o8s2EcfGlPfR&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005003392263399.html?spm=a2g0o.productlist.main.43.69a766bfTV5HDi&algo_pvid=bfaa59b8-ce29-490e-9f21-f8b2c866d5d7&algo_exp_id=bfaa59b8-ce29-490e-9f21-f8b2c866d5d7-21&pdp_ext_f=%7B%22order%22%3A%225060%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21615.89%21492.68%21%21%2136.64%2129.31%21%402101062a17404578228845116e495c%2112000028674050699%21sea%21BD%210%21ABX&curPageLogUid=ICIx8VMTvc5w&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006344566079.html?spm=a2g0o.productlist.main.105.69a766bfTV5HDi&algo_pvid=bfaa59b8-ce29-490e-9f21-f8b2c866d5d7&algo_exp_id=bfaa59b8-ce29-490e-9f21-f8b2c866d5d7-52&pdp_ext_f=%7B%22order%22%3A%227131%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211046.20%21491.66%21%21%2162.24%2129.25%21%402101062a17404578228845116e495c%2112000036832923101%21sea%21BD%210%21ABX&curPageLogUid=0Yb1zO1YP7Wr&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005956298915.html?aem_p4p_detail=202502242037066039541144993630000098064&algo_pvid=89d19c93-426f-4e1c-896c-66cd39f0482f&algo_exp_id=89d19c93-426f-4e1c-896c-66cd39f0482f-3&pdp_ext_f=%7B%22order%22%3A%227108%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21840.46%21661.45%21%21%2150.00%2139.35%21%4021015b7d17404582262217695e3a25%2112000035018762071%21sea%21BD%210%21ABX&curPageLogUid=k5gx895CnFes&utparam-url=scene%3Asearch%7Cquery_from%3A&search_p4p_id=202502242037066039541144993630000098064_1"
"https://www.aliexpress.com/item/1005006435587196.html?spm=a2g0o.productlist.main.111.2361rdGMrdGMJk&algo_pvid=907d6cd4-449c-4bde-9378-44179836a81b&algo_exp_id=907d6cd4-449c-4bde-9378-44179836a81b-55&pdp_ext_f=%7B%22order%22%3A%226067%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21312.99%21140.86%21%21%2118.62%218.38%21%402101584917404583016672206e4694%2112000037156993494%21sea%21BD%210%21ABX&curPageLogUid=Wa6or5uBuef4&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007445128236.html?spm=a2g0o.productlist.main.1.759b77f1lI9D3v&algo_pvid=dd395be7-de16-4051-bc3b-d36707505a12&algo_exp_id=dd395be7-de16-4051-bc3b-d36707505a12-0&pdp_ext_f=%7B%22order%22%3A%224323%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%213106.96%211553.48%21%21%2125.60%2112.80%21%40210123bc17404584084918103eee7f%2112000040779747394%21sea%21BD%210%21ABX&curPageLogUid=NkjW4DyVHZ0R&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007446278321.html?spm=a2g0o.productlist.main.3.759b77f1lI9D3v&algo_pvid=dd395be7-de16-4051-bc3b-d36707505a12&algo_exp_id=dd395be7-de16-4051-bc3b-d36707505a12-1&pdp_ext_f=%7B%22order%22%3A%221245%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212864.23%211432.11%21%21%2123.60%2111.80%21%40210123bc17404584084918103eee7f%2112000040784085207%21sea%21BD%210%21ABX&curPageLogUid=nqhCIdLvCEEj&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006876657863.html?spm=a2g0o.productlist.main.59.4b69gkUxgkUxvD&algo_pvid=8731155b-c0ee-4eed-83e9-1afae48d32ef&algo_exp_id=8731155b-c0ee-4eed-83e9-1afae48d32ef-29&pdp_ext_f=%7B%22order%22%3A%225002%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%214899.38%213282.50%21%21%21291.47%21195.28%21%40214100f417404592982304776e46e7%2112000038593548803%21sea%21BD%210%21ABX&curPageLogUid=vfikaabyjAlR&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/32549161202.html?spm=a2g0o.productlist.main.3.135cM0bvM0bvW2&aem_p4p_detail=20250225081014711342804637140001366481&algo_pvid=77d585b2-f9ee-47e4-b971-b967e43fdfbc&algo_exp_id=77d585b2-f9ee-47e4-b971-b967e43fdfbc-1&pdp_ext_f=%7B%22order%22%3A%222194%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%215540.35%212049.87%21%21%2145.65%2116.89%21%40%2112000026753986886%21sea%21BD%210%21ABX&curPageLogUid=nTXP0eZWdgkr&utparam-url=scene%3Asearch%7Cquery_from%3A&search_p4p_id=20250225081014711342804637140001366481_2",
"https://www.aliexpress.com/item/1005006230201573.html?aem_p4p_detail=202502250811109435933449177040001322345&algo_pvid=fcea4d6a-3f87-4bcc-9dc2-0b4078db1317&algo_exp_id=fcea4d6a-3f87-4bcc-9dc2-0b4078db1317-0&pdp_ext_f=%7B%22order%22%3A%221102%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2117496.09%218573.28%21%21%21144.16%2170.64%21%40%2112000036382645846%21sea%21BD%210%21ABX&curPageLogUid=dODdGjPnCsvJ&utparam-url=scene%3Asearch%7Cquery_from%3A&search_p4p_id=202502250811109435933449177040001322345_1",
"https://www.aliexpress.com/item/1005006470108540.html?spm=a2g0o.productlist.main.117.135cnHoOnHoOlD&algo_pvid=fcea4d6a-3f87-4bcc-9dc2-0b4078db1317&algo_exp_id=fcea4d6a-3f87-4bcc-9dc2-0b4078db1317-58&pdp_ext_f=%7B%22order%22%3A%22913%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2110920.49%214040.26%21%21%2189.98%2133.29%21%40%2112000037315314075%21sea%21BD%210%21ABX&curPageLogUid=adFPsZcdc9Uu&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005742938199.html?spm=a2g0o.productlist.main.33.742325fG25fGZB&algo_pvid=704d447e-d046-4420-b726-5f3d968d8157&algo_exp_id=704d447e-d046-4420-b726-5f3d968d8157-16&pdp_ext_f=%7B%22order%22%3A%223898%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2111163.23%213348.49%21%21%2191.98%2127.59%21%40%2112000034264130572%21sea%21BD%210%21ABX&curPageLogUid=w3EUPUU1RHpo&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006451692571.html?spm=a2g0o.productlist.main.41.742325fG25fGZB&algo_pvid=704d447e-d046-4420-b726-5f3d968d8157&algo_exp_id=704d447e-d046-4420-b726-5f3d968d8157-20&pdp_ext_f=%7B%22order%22%3A%221123%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%215073.09%212892.15%21%21%2141.80%2123.83%21%40%2112000037223004996%21sea%21BD%210%21ABX&curPageLogUid=jDLNFl3YUSwk&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006448748697.html?spm=a2g0o.productlist.main.57.742325fG25fGZB&algo_pvid=704d447e-d046-4420-b726-5f3d968d8157&algo_exp_id=704d447e-d046-4420-b726-5f3d968d8157-28&pdp_ext_f=%7B%22order%22%3A%223349%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%215337.67%212668.84%21%21%2143.98%2121.99%21%40%2112000037208258597%21sea%21BD%210%21ABX&curPageLogUid=kyzwIFt6ta2D&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006333819207.html?spm=a2g0o.productlist.main.61.742325fG25fGZB&algo_pvid=704d447e-d046-4420-b726-5f3d968d8157&algo_exp_id=704d447e-d046-4420-b726-5f3d968d8157-30&pdp_ext_f=%7B%22order%22%3A%221222%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212284.10%211256.13%21%21%2118.82%2110.35%21%40%2112000036795998773%21sea%21BD%210%21ABX&curPageLogUid=ZU1af4bohQ4r&utparam-url=scene%3Asearch%7Cquery_from%3A",
]

print(len(urls))

41


In [ ]:
max_feedbacks = ["784","274","248","1964","899","523","304","362","529","1323","557","225","703","577","341","296","435","645","296","377","417","335","1072","2414","660","582","450","470","691","977","870","700","1087","453","785","483","233","202","758","236","606","265"]

print(len(max_feedbacks))

42


In [ ]:
# Run the scraper for all URLs and save to a single CSV
scrape_multiple_urls(urls, max_feedbacks)



🔗 URL 1/41: https://www.aliexpress.com/item/1005006876657863.html?spm=a2g0o.productlist.main.29.4b692d3c6MUfau&algo_pvid=567885e4-a76a-4d6a-84d6-b4830f6e63f1&algo_exp_id=567885e4-a76a-4d6a-84d6-b4830f6e63f1-14&pdp_ext_f=%7B%22order%22%3A%224997%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%214913.66%213292.07%21%21%21291.47%21195.28%21%402141122217404120397816133ee933%2112000038593548803%21sea%21BD%210%21ABX&curPageLogUid=VYRRl95NIhMF&utparam-url=scene%3Asearch%7Cquery_from%3A
📝 Number of feedbacks collected from this URL: 741

🔗 URL 2/41: https://www.aliexpress.com/item/1005006513094908.html?spm=a2g0o.productlist.main.9.60111e92KYhopW&algo_pvid=0117ce36-3fd2-45ec-8439-aba6cf381615&algo_exp_id=0117ce36-3fd2-45ec-8439-aba6cf381615-4&pdp_ext_f=%7B%22order%22%3A%221327%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211215.44%21607.72%21%21%2110.00%215.00%21%402141001d17404149920752797ef4d0%2112000037484312460%21sea%21BD%210%21ABX&curPageLogUid=I8fzcWrcW9n4&utparam-url=scene%3Asea

2nd phase Scraping

In [ ]:
urls2=["https://www.aliexpress.com/item/1005005402456919.html?spm=a2g0o.productlist.main.83.742325fG25fGZB&algo_pvid=704d447e-d046-4420-b726-5f3d968d8157&algo_exp_id=704d447e-d046-4420-b726-5f3d968d8157-41&pdp_ext_f=%7B%22order%22%3A%221039%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21756.25%21226.93%21%21%2144.99%2113.50%21%40%2112000032916107016%21sea%21BD%210%21ABX&curPageLogUid=h6HlQLz4cRPh&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006203270165.html?spm=a2g0o.productlist.main.35.202277b162y51G&algo_pvid=08c0a288-7ebd-4e9b-be3f-9d0ffa420171&algo_exp_id=08c0a288-7ebd-4e9b-be3f-9d0ffa420171-17&pdp_ext_f=%7B%22order%22%3A%221168%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21463.62%21236.67%21%21%213.82%211.95%21%40%2112000036254316046%21sea%21BD%210%21ABX&curPageLogUid=2fGTH2RCGnXs&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007007013772.html?spm=a2g0o.productlist.main.47.202277b162y51G&algo_pvid=08c0a288-7ebd-4e9b-be3f-9d0ffa420171&algo_exp_id=08c0a288-7ebd-4e9b-be3f-9d0ffa420171-23&pdp_ext_f=%7B%22order%22%3A%221955%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%217973.73%214784.24%21%21%2165.70%2139.42%21%40%2112000042215394544%21sea%21BD%210%21ABX&curPageLogUid=rFdOo6gL4XK3&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006851800028.html?spm=a2g0o.productlist.main.29.3ac6nNWtnNWtoT&algo_pvid=336b263c-820f-4dbe-b3ac-c12c8df660ed&algo_exp_id=336b263c-820f-4dbe-b3ac-c12c8df660ed-14&pdp_ext_f=%7B%22order%22%3A%22794%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2122537.63%2114198.59%21%21%21185.70%21116.99%21%40%2112000038514909953%21sea%21BD%210%21ABX&curPageLogUid=eEbns9ys8wUR&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006979228312.html?spm=a2g0o.productlist.main.29.3ac6nNWtnNWtoT&algo_pvid=99e05392-f0a9-4151-b36a-c7a943eb1532&algo_exp_id=99e05392-f0a9-4151-b36a-c7a943eb1532-14&pdp_ext_f=%7B%22order%22%3A%221040%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2112134.15%213639.76%21%21%2199.98%2129.99%21%40%2112000041141049335%21sea%21BD%210%21ABX&curPageLogUid=RJo8wKm1rVxd&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007369344549.html?spm=a2g0o.productlist.main.55.3ac6nNWtnNWtoT&algo_pvid=a730bb6b-589e-494c-9088-2cbc420ededa&algo_exp_id=a730bb6b-589e-494c-9088-2cbc420ededa-27&pdp_ext_f=%7B%22order%22%3A%222042%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2112134.15%213275.66%21%21%2199.98%2126.99%21%40%2112000044697930462%21sea%21BD%210%21ABX&curPageLogUid=LLdVTpGM5c5P&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006075900919.html?spm=a2g0o.productlist.main.87.3ac6nNWtnNWtoT&algo_pvid=8c2d6311-286c-4056-81f5-ec5d9d195a5a&algo_exp_id=8c2d6311-286c-4056-81f5-ec5d9d195a5a-43&pdp_ext_f=%7B%22order%22%3A%222694%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2117786.32%212845.80%21%21%211058.13%21169.30%21%40%2112000035616860156%21sea%21BD%210%21ABX&curPageLogUid=TPyxFqQMsoie&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006620616867.html?spm=a2g0o.productlist.main.83.3ac6nNWtnNWtoT&algo_pvid=8c2d6311-286c-4056-81f5-ec5d9d195a5a&algo_exp_id=8c2d6311-286c-4056-81f5-ec5d9d195a5a-41&pdp_ext_f=%7B%22order%22%3A%22971%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%216065.86%213579.07%21%21%2149.98%2129.49%21%40%2112000041068142697%21sea%21BD%210%21ABX&curPageLogUid=C8O3Krz6w4p0&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005002939086341.html?spm=a2g0o.productlist.main.95.54a6BlyNBlyNMQ&algo_pvid=f6483041-0496-4710-944b-04d3e72c5deb&algo_exp_id=f6483041-0496-4710-944b-04d3e72c5deb-47&pdp_ext_f=%7B%22order%22%3A%22738%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212139.68%211284.05%21%21%2117.63%2110.58%21%40%2112000040713497435%21sea%21BD%210%21ABX&curPageLogUid=SEQfoERLiBxH&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005288989392.html?spm=a2g0o.productlist.main.51.54a6BlyNBlyNMQ&algo_pvid=ab369b3e-8171-4e38-b6b1-fb543eb07c61&algo_exp_id=ab369b3e-8171-4e38-b6b1-fb543eb07c61-25&pdp_ext_f=%7B%22order%22%3A%22662%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21296.13%21133.50%21%21%212.44%211.10%21%40%2112000032503790767%21sea%21BD%210%21ABX&curPageLogUid=xKdtamv3HKEp&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005003227386920.html?spm=a2g0o.productlist.main.31.48c15073PurjWV&algo_pvid=63222404-79da-42d3-ad33-4f870f692ff0&algo_exp_id=63222404-79da-42d3-ad33-4f870f692ff0-15&pdp_ext_f=%7B%22order%22%3A%225289%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211905.44%211333.81%21%21%2115.70%2110.99%21%40%2112000042934966591%21sea%21BD%210%21ABX&curPageLogUid=iWAZYkbVSMyq&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004696774991.html?spm=a2g0o.productlist.main.19.48c15073PurjWV&algo_pvid=63222404-79da-42d3-ad33-4f870f692ff0&algo_exp_id=63222404-79da-42d3-ad33-4f870f692ff0-9&pdp_ext_f=%7B%22order%22%3A%225376%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212079.00%211309.54%21%21%2117.13%2110.79%21%40%2112000035214348309%21sea%21BD%210%21ABX&curPageLogUid=kZKQGIgj4WPA&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005992910426.html?spm=a2g0o.productlist.main.17.48c15073PurjWV&algo_pvid=63222404-79da-42d3-ad33-4f870f692ff0&algo_exp_id=63222404-79da-42d3-ad33-4f870f692ff0-8&pdp_ext_f=%7B%22order%22%3A%221423%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212945.55%211898.16%21%21%2124.27%2115.64%21%40%2112000042930095245%21sea%21BD%210%21ABX&curPageLogUid=8sFwOeppdkMq&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007331274299.html?aem_p4p_detail=202502250835513598891360280370001236976&algo_pvid=63222404-79da-42d3-ad33-4f870f692ff0&algo_exp_id=63222404-79da-42d3-ad33-4f870f692ff0-1&pdp_ext_f=%7B%22order%22%3A%221521%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21655.56%21439.23%21%21%2139.00%2126.13%21%40%2112000040317737547%21sea%21BD%210%21ABX&curPageLogUid=oJaLijRqHQoc&utparam-url=scene%3Asearch%7Cquery_from%3A&search_p4p_id=202502250835513598891360280370001236976_2",
"https://www.aliexpress.com/item/1005007628910870.html?spm=a2g0o.productlist.main.95.8b9bI8jdI8jdCq&algo_pvid=ca566fad-d02c-4347-a9dc-ba34f7adbb0c&algo_exp_id=ca566fad-d02c-4347-a9dc-ba34f7adbb0c-47&pdp_ext_f=%7B%22order%22%3A%2213069%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212060.79%211030.39%21%21%2116.98%218.49%21%40%2112000041566887451%21sea%21BD%210%21ABX&curPageLogUid=lbdWLo73NOga&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006195931488.html?spm=a2g0o.productlist.main.89.8b9bI8jdI8jdCq&algo_pvid=ca566fad-d02c-4347-a9dc-ba34f7adbb0c&algo_exp_id=ca566fad-d02c-4347-a9dc-ba34f7adbb0c-44&pdp_ext_f=%7B%22order%22%3A%221368%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21726.98%21508.52%21%21%215.99%214.19%21%40%2112000036221245998%21sea%21BD%210%21ABX&curPageLogUid=rB6qus4R4NJl&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007614008734.html?spm=a2g0o.productlist.main.75.8b9bI8jdI8jdCq&algo_pvid=ca566fad-d02c-4347-a9dc-ba34f7adbb0c&algo_exp_id=ca566fad-d02c-4347-a9dc-ba34f7adbb0c-37&pdp_ext_f=%7B%22order%22%3A%2211087%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211091.08%21544.93%21%21%218.99%214.49%21%40%2112000041509687091%21sea%21BD%210%21ABX&curPageLogUid=JfqwYgD3k4w7&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005992910426.html?spm=a2g0o.productlist.main.37.8b9bI8jdI8jdCq&algo_pvid=ca566fad-d02c-4347-a9dc-ba34f7adbb0c&algo_exp_id=ca566fad-d02c-4347-a9dc-ba34f7adbb0c-18&pdp_ext_f=%7B%22order%22%3A%221423%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212945.55%211898.16%21%21%2124.27%2115.64%21%40%2112000042930095245%21sea%21BD%210%21ABX&curPageLogUid=dRJhaeN1EbVb&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005467455219.html?spm=a2g0o.productlist.main.109.8b9bI8jdI8jdCq&algo_pvid=330e3de3-f3e6-4e72-9ff9-effe5515db69&algo_exp_id=330e3de3-f3e6-4e72-9ff9-effe5515db69-54&pdp_ext_f=%7B%22order%22%3A%225381%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21909.03%21669.94%21%21%217.49%215.52%21%40%2112000042934069908%21sea%21BD%210%21ABX&curPageLogUid=YyAqTQffjSmN&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004696774991.html?spm=a2g0o.productlist.main.27.8b9bI8jdI8jdCq&algo_pvid=330e3de3-f3e6-4e72-9ff9-effe5515db69&algo_exp_id=330e3de3-f3e6-4e72-9ff9-effe5515db69-13&pdp_ext_f=%7B%22order%22%3A%225376%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%212079.00%211309.54%21%21%2117.13%2110.79%21%40%2112000035214348309%21sea%21BD%210%21ABX&curPageLogUid=74oUToikZMZg&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006016653553.html?spm=a2g0o.productlist.main.19.8b9bI8jdI8jdCq&algo_pvid=330e3de3-f3e6-4e72-9ff9-effe5515db69&algo_exp_id=330e3de3-f3e6-4e72-9ff9-effe5515db69-9&pdp_ext_f=%7B%22order%22%3A%221230%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211384.78%21872.62%21%21%2111.41%217.19%21%40%2112000041705492845%21sea%21BD%210%21ABX&curPageLogUid=gjWWLtVQYad3&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006398654272.html?spm=a2g0o.productlist.main.61.2b0231c3dlNl5F&algo_pvid=873911cf-15eb-47b2-b14f-9e121c70bedb&algo_exp_id=873911cf-15eb-47b2-b14f-9e121c70bedb-30&pdp_ext_f=%7B%22order%22%3A%222483%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%217453.07%214695.64%21%21%2161.41%2138.69%21%40%2112000044698684045%21sea%21BD%210%21ABX&curPageLogUid=g6QHB6gwHeCD&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004651662970.html?spm=a2g0o.productlist.main.115.2b0231c3dlNl5F&algo_pvid=f2f6602b-2f40-4259-b487-18c18c495eaa&algo_exp_id=f2f6602b-2f40-4259-b487-18c18c495eaa-57&pdp_ext_f=%7B%22order%22%3A%221716%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21462.40%21277.92%21%21%213.81%212.29%21%40%2112000029983471575%21sea%21BD%210%21ABX&curPageLogUid=mPKUadAfMcuz&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/32951056936.html?spm=a2g0o.productlist.main.107.2b0231c3dlNl5F&algo_pvid=f2f6602b-2f40-4259-b487-18c18c495eaa&algo_exp_id=f2f6602b-2f40-4259-b487-18c18c495eaa-53&pdp_ext_f=%7B%22order%22%3A%22792%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2199.52%2174.03%21%21%210.82%210.61%21%40%2167126618572%21sea%21BD%210%21ABX&curPageLogUid=3ZRY3FSBbj67&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/4000147858257.html?spm=a2g0o.productlist.main.89.2b0231c3dlNl5F&algo_pvid=f2f6602b-2f40-4259-b487-18c18c495eaa&algo_exp_id=f2f6602b-2f40-4259-b487-18c18c495eaa-44&pdp_ext_f=%7B%22order%22%3A%22600%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2121.85%2114.57%21%21%210.18%210.12%21%40%2110000000453267008%21sea%21BD%210%21ABX&curPageLogUid=7b6sLMtgCQd1&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006071611096.html?spm=a2g0o.productlist.main.57.2b0231c3dlNl5F&algo_pvid=f2f6602b-2f40-4259-b487-18c18c495eaa&algo_exp_id=f2f6602b-2f40-4259-b487-18c18c495eaa-28&pdp_ext_f=%7B%22order%22%3A%22647%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21577.23%21357.87%21%21%2134.34%2121.29%21%40%2112000035595938407%21sea%21BD%210%21ABX&curPageLogUid=13U07mZMqce7&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006756225772.html?spm=a2g0o.productlist.main.21.2b0231c3dlNl5F&algo_pvid=f2f6602b-2f40-4259-b487-18c18c495eaa&algo_exp_id=f2f6602b-2f40-4259-b487-18c18c495eaa-10&pdp_ext_f=%7B%22order%22%3A%22884%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211699.12%211036.46%21%21%2114.00%218.54%21%40%2112000045556541052%21sea%21BD%210%21ABX&curPageLogUid=lum4pQRKykC4&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004211225407.html?spm=a2g0o.productlist.main.11.3ac839abumUhSF&algo_pvid=1ccf3a7d-e5ab-4e8a-8818-a01f8ed5dcf8&algo_exp_id=1ccf3a7d-e5ab-4e8a-8818-a01f8ed5dcf8-5&pdp_ext_f=%7B%22order%22%3A%222763%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2170.39%2135.19%21%21%210.58%210.29%21%40%2112000028394187436%21sea%21BD%210%21ABX&curPageLogUid=TOTmdccdjJFC&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007806220676.html?spm=a2g0o.productlist.main.1.3ac839abumUhSF&algo_pvid=1ccf3a7d-e5ab-4e8a-8818-a01f8ed5dcf8&algo_exp_id=1ccf3a7d-e5ab-4e8a-8818-a01f8ed5dcf8-0&pdp_ext_f=%7B%22order%22%3A%224462%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21141.20%21139.86%21%21%218.40%218.32%21%40%2112000042264485133%21sea%21BD%210%21ABX&curPageLogUid=jxjHJd4qNmTG&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006519144465.html?spm=a2g0o.productlist.main.17.77bb4a6s4a6sSQ&algo_pvid=93bae6bc-9168-4ce5-9b10-474945e0b4d4&algo_exp_id=93bae6bc-9168-4ce5-9b10-474945e0b4d4-8&pdp_ext_f=%7B%22order%22%3A%223131%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21140.78%2184.95%21%21%211.16%210.70%21%40%2112000037511304699%21sea%21BD%210%21ABX&curPageLogUid=wKE78b1v0NPR&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005191428706.html?spm=a2g0o.productlist.main.53.77bb4a6s4a6sSQ&algo_pvid=ee0c3663-36d2-4461-9f6c-caad6328fb2c&algo_exp_id=ee0c3663-36d2-4461-9f6c-caad6328fb2c-26&pdp_ext_f=%7B%22order%22%3A%222861%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21546.15%21109.23%21%21%214.50%210.90%21%40%2112000032062198957%21sea%21BD%210%21ABX&curPageLogUid=MwznhTnlP5Eh&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007005909112.html?spm=a2g0o.productlist.main.83.77bb4a6s4a6sSQ&algo_pvid=ee0c3663-36d2-4461-9f6c-caad6328fb2c&algo_exp_id=ee0c3663-36d2-4461-9f6c-caad6328fb2c-41&pdp_ext_f=%7B%22order%22%3A%222206%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21302.57%21181.54%21%21%2118.00%2110.80%21%40%2112000039030536136%21sea%21BD%210%21ABX&curPageLogUid=942R2IghG3kf&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004775106320.html?spm=a2g0o.productlist.main.93.77bb4a6s4a6sSQ&algo_pvid=ee0c3663-36d2-4461-9f6c-caad6328fb2c&algo_exp_id=ee0c3663-36d2-4461-9f6c-caad6328fb2c-46&pdp_ext_f=%7B%22order%22%3A%22828%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21192.97%21160.20%21%21%211.59%211.32%21%40%2112000030431946433%21sea%21BD%210%21ABX&curPageLogUid=1QFkuumjF3Kc&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006779056175.html?spm=a2g0o.productlist.main.59.694a408cVu06KE&algo_pvid=b79d9de5-cf31-454c-932d-6306b9610ce6&algo_exp_id=b79d9de5-cf31-454c-932d-6306b9610ce6-29&pdp_ext_f=%7B%22order%22%3A%221329%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2194.67%2175.25%21%21%210.78%210.62%21%40%2112000038269749787%21sea%21BD%210%21ABX&curPageLogUid=jBgAwMBaSqKM&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005393085350.html?spm=a2g0o.productlist.main.27.694a408cVu06KE&algo_pvid=b79d9de5-cf31-454c-932d-6306b9610ce6&algo_exp_id=b79d9de5-cf31-454c-932d-6306b9610ce6-13&pdp_ext_f=%7B%22order%22%3A%221854%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21241.52%21168.70%21%21%211.99%211.39%21%40%2112000032876038874%21sea%21BD%210%21ABX&curPageLogUid=XhiqrjtBlvva&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007641494414.html?spm=a2g0o.productlist.main.5.694a408cVu06KE&algo_pvid=b79d9de5-cf31-454c-932d-6306b9610ce6&algo_exp_id=b79d9de5-cf31-454c-932d-6306b9610ce6-2&pdp_ext_f=%7B%22order%22%3A%221490%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21117.66%2158.83%21%21%217.00%213.50%21%40%2112000041614619736%21sea%21BD%210%21ABX&curPageLogUid=bqPalvc8PqNZ&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006514697259.html?spm=a2g0o.productlist.main.9.694a408cVu06KE&algo_pvid=b79d9de5-cf31-454c-932d-6306b9610ce6&algo_exp_id=b79d9de5-cf31-454c-932d-6306b9610ce6-4&pdp_ext_f=%7B%22order%22%3A%22874%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21293.71%21149.28%21%21%212.42%211.23%21%40%2112000037490579035%21sea%21BD%210%21ABX&curPageLogUid=5zfTRcR2yMM0&utparam-url=scene%3Asearch%7Cquery_from%3A"
]

print(len(urls2))

37


In [ ]:
max_feedbacks2=["210","230","407","316","394","1052","787","258","134","141","845","979","209","225","1818","275","1396","209","1187","979","244","394","307","200","200","172","198","542","1192","583","475","310","154","440","659","307","206"]
print(len(max_feedbacks2))

37


In [ ]:
# Run the scraper for all URLs and save to a single CSV
scrape_multiple_urls(urls2, max_feedbacks2)

🔗 URL 1/37: https://www.aliexpress.com/item/1005005402456919.html?spm=a2g0o.productlist.main.83.742325fG25fGZB&algo_pvid=704d447e-d046-4420-b726-5f3d968d8157&algo_exp_id=704d447e-d046-4420-b726-5f3d968d8157-41&pdp_ext_f=%7B%22order%22%3A%221039%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21756.25%21226.93%21%21%2144.99%2113.50%21%40%2112000032916107016%21sea%21BD%210%21ABX&curPageLogUid=h6HlQLz4cRPh&utparam-url=scene%3Asearch%7Cquery_from%3A
📝 Number of feedbacks collected from this URL: 210

🔗 URL 2/37: https://www.aliexpress.com/item/1005006203270165.html?spm=a2g0o.productlist.main.35.202277b162y51G&algo_pvid=08c0a288-7ebd-4e9b-be3f-9d0ffa420171&algo_exp_id=08c0a288-7ebd-4e9b-be3f-9d0ffa420171-17&pdp_ext_f=%7B%22order%22%3A%221168%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21463.62%21236.67%21%21%213.82%211.95%21%40%2112000036254316046%21sea%21BD%210%21ABX&curPageLogUid=2fGTH2RCGnXs&utparam-url=scene%3Asearch%7Cquery_from%3A
📝 Number of feedbacks collected from this UR

KeyboardInterrupt: 

3rd phase Scraping

In [ ]:
url3=["https://www.aliexpress.com/item/1005006471222041.html?spm=a2g0o.productlist.main.115.4bbb1611UQA8lu&algo_pvid=0312d218-e564-4d6a-875b-ed9162f5814c&algo_exp_id=0312d218-e564-4d6a-875b-ed9162f5814c-57&pdp_ext_f=%7B%22order%22%3A%227416%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21833.74%21458.07%21%21%216.88%213.78%21%40210141f717412384359201548eaffc%2112000037317581168%21sea%21BD%210%21ABX&curPageLogUid=BY8QaYj8G0m5&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005002908009280.html?spm=a2g0o.productlist.main.41.4bbb1611UQA8lu&algo_pvid=97c93f81-7324-45d8-ac55-5c01c670525a&algo_exp_id=97c93f81-7324-45d8-ac55-5c01c670525a-20&pdp_ext_f=%7B%22order%22%3A%223127%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21325.98%21208.43%21%21%212.69%211.72%21%402101062a17412384987491595ec517%2112000022722852421%21sea%21BD%210%21ABX&curPageLogUid=5MyXzCpqzDx9&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007441603612.html?spm=a2g0o.productlist.main.49.4bbb1611UQA8lu&algo_pvid=97c93f81-7324-45d8-ac55-5c01c670525a&algo_exp_id=97c93f81-7324-45d8-ac55-5c01c670525a-24&pdp_ext_f=%7B%22order%22%3A%222963%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21393.85%21180.57%21%21%213.25%211.49%21%402101062a17412384987491595ec517%2112000040769630192%21sea%21BD%210%21ABX&curPageLogUid=KLMRitwE44cX&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005008320078821.html?spm=a2g0o.productlist.main.109.4bbb1611UQA8lu&algo_pvid=97c93f81-7324-45d8-ac55-5c01c670525a&algo_exp_id=97c93f81-7324-45d8-ac55-5c01c670525a-54&pdp_ext_f=%7B%22order%22%3A%227216%22%2C%22eval%22%3A%221%22%2C%22orig_sl_item_id%22%3A%221005008320078821%22%2C%22orig_item_id%22%3A%221005006295902116%22%7D&pdp_npi=4%40dis%21BDT%214645.40%212322.70%21%21%21276.18%21138.09%21%402101062a17412384987491595ec517%2112000044596483454%21sea%21BD%210%21ABX&curPageLogUid=u9BfeO210ArA&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005008051814219.html?spm=a2g0o.productlist.main.113.4bbb1611UQA8lu&algo_pvid=97c93f81-7324-45d8-ac55-5c01c670525a&algo_exp_id=97c93f81-7324-45d8-ac55-5c01c670525a-56&pdp_ext_f=%7B%22order%22%3A%222227%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211173.05%21586.52%21%21%219.68%214.84%21%402101062a17412384987491595ec517%2112000043448180426%21sea%21BD%210%21ABX&curPageLogUid=a541d4HurRAw&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004282025592.html?spm=a2g0o.productlist.main.25.4bbb1611UQA8lu&algo_pvid=889be7d7-c9b5-4ec8-86ee-3d45d6930b40&algo_exp_id=889be7d7-c9b5-4ec8-86ee-3d45d6930b40-12&pdp_ext_f=%7B%22order%22%3A%221001%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21821.62%21369.61%21%21%216.78%213.05%21%402101246417412385385455052e78d2%2112000028617135693%21sea%21BD%210%21ABX&curPageLogUid=OsDCdJroNHH9&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005998253586.html?spm=a2g0o.productlist.main.61.4bbb1611UQA8lu&algo_pvid=889be7d7-c9b5-4ec8-86ee-3d45d6930b40&algo_exp_id=889be7d7-c9b5-4ec8-86ee-3d45d6930b40-30&pdp_ext_f=%7B%22order%22%3A%221069%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21478.67%21238.73%21%21%213.95%211.97%21%402101246417412385385455052e78d2%2112000035238061238%21sea%21BD%210%21ABX&curPageLogUid=hkKqabe4iR6Y&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004493682356.html?spm=a2g0o.productlist.main.1.117a6f02PFPu3f&algo_pvid=68d2d61d-82f3-4393-a277-843b3b3d932b&algo_exp_id=68d2d61d-82f3-4393-a277-843b3b3d932b-0&pdp_ext_f=%7B%22order%22%3A%221626%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211373.01%21425.36%21%21%2111.33%213.51%21%40213ba8cc17412391942338498e724b%2112000029351807704%21sea%21BD%210%21ABX&curPageLogUid=7iOGvQDEgM55&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007463095132.html?spm=a2g0o.productlist.main.17.117a6f02PFPu3f&algo_pvid=68d2d61d-82f3-4393-a277-843b3b3d932b&algo_exp_id=68d2d61d-82f3-4393-a277-843b3b3d932b-8&pdp_ext_f=%7B%22order%22%3A%221709%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21704.43%21352.22%21%21%2141.88%2120.94%21%40213ba8cc17412391942338498e724b%2112000040856433518%21sea%21BD%210%21ABX&curPageLogUid=DNlsro9z2PiV&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007295410102.html?spm=a2g0o.productlist.main.89.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-44&pdp_ext_f=%7B%22order%22%3A%221073%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21474.83%21237.33%21%21%2128.23%2114.11%21%4021015b2417412391480291891ec674%2112000040098698915%21sea%21BD%210%21ABX&curPageLogUid=15OnT5EiWkBv&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004583790156.html?spm=a2g0o.productlist.main.65.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-32&pdp_ext_f=%7B%22order%22%3A%225013%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211085.80%21455.65%21%21%218.96%213.76%21%4021015b2417412391480291891ec674%2112000029717611620%21sea%21BD%210%21ABX&curPageLogUid=mwBbCYnHJcsZ&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006744557765.html?spm=a2g0o.productlist.main.61.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-30&pdp_ext_f=%7B%22order%22%3A%22922%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21482.31%21265.39%21%21%213.98%212.19%21%4021015b2417412391480291891ec674%2112000038163940372%21sea%21BD%210%21ABX&curPageLogUid=6Gb6VHtkLdwi&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007607916542.html?spm=a2g0o.productlist.main.53.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-26&pdp_ext_f=%7B%22order%22%3A%22744%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21437.47%21241.15%21%21%213.61%211.99%21%4021015b2417412391480291891ec674%2112000041487574268%21sea%21BD%210%21ABX&curPageLogUid=aeWMQxPqEy2n&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007138972467.html?spm=a2g0o.productlist.main.41.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-20&pdp_ext_f=%7B%22order%22%3A%221589%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21531.99%21271.45%21%21%214.39%212.24%21%4021015b2417412391480291891ec674%2112000039541753405%21sea%21BD%210%21ABX&curPageLogUid=RVmfgcaEv7QJ&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006509260259.html?spm=a2g0o.productlist.main.35.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-17&pdp_ext_f=%7B%22order%22%3A%22860%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%213102.49%211023.85%21%21%21184.45%2160.87%21%4021015b2417412391480291891ec674%2112000037459985944%21sea%21BD%210%21ABX&curPageLogUid=sgygIhYNC0Ap&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005004459905655.html?spm=a2g0o.productlist.main.1.117a6f02PFPu3f&algo_pvid=7583c9ef-5c47-4861-b63f-0171c9538456&algo_exp_id=7583c9ef-5c47-4861-b63f-0171c9538456-0&pdp_ext_f=%7B%22order%22%3A%221658%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%211484.49%21444.74%21%21%2112.25%213.67%21%4021015b2417412391480291891ec674%2112000029254247403%21sea%21BD%210%21ABX&curPageLogUid=3VvvuD8TJnRh&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005300580064.html?spm=a2g0o.productlist.main.9.5f954638rbJEJt&algo_pvid=b57e4977-d977-4d86-b53b-a86b08260497&algo_exp_id=b57e4977-d977-4d86-b53b-a86b08260497-4&pdp_ext_f=%7B%22order%22%3A%221810%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21117.24%2182.09%21%21%216.97%214.88%21%40210141f717412391107926185eaffb%2112000032546606359%21sea%21BD%210%21ABX&curPageLogUid=YOxFcMMSLCDd&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006036650619.html?spm=a2g0o.productlist.main.107.705a6166mYNOyn&algo_pvid=6644bb34-e02f-41ec-a277-71c920914fd9&algo_exp_id=6644bb34-e02f-41ec-a277-71c920914fd9-53&pdp_ext_f=%7B%22order%22%3A%223700%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21157.61%21103.95%21%21%219.37%216.18%21%402101062a17412397821191011ec531%2112000035426722554%21sea%21BD%210%21ABX&curPageLogUid=7G3NxMGCTcVV&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006421446826.html?spm=a2g0o.productlist.main.81.705a6166mYNOyn&algo_pvid=358d0cdf-fc7f-4ca3-9414-83ea9ddc11c5&algo_exp_id=358d0cdf-fc7f-4ca3-9414-83ea9ddc11c5-40&pdp_ext_f=%7B%22order%22%3A%2216293%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2122.71%2118.00%21%21%211.35%211.07%21%402101584917412397443164088ed614%2112000037104507857%21sea%21BD%210%21ABX&curPageLogUid=f3XO5vGYC7AQ&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006782366605.html?spm=a2g0o.productlist.main.69.705a6166mYNOyn&algo_pvid=358d0cdf-fc7f-4ca3-9414-83ea9ddc11c5&algo_exp_id=358d0cdf-fc7f-4ca3-9414-83ea9ddc11c5-34&pdp_ext_f=%7B%22order%22%3A%222532%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2151.47%2133.47%21%21%213.06%211.99%21%402101584917412397443164088ed614%2112000038280746053%21sea%21BD%210%21ABX&curPageLogUid=DBpylMxYcj3i&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007082552825.html?spm=a2g0o.productlist.main.1.705a6166mYNOyn&algo_pvid=358d0cdf-fc7f-4ca3-9414-83ea9ddc11c5&algo_exp_id=358d0cdf-fc7f-4ca3-9414-83ea9ddc11c5-0&pdp_ext_f=%7B%22order%22%3A%221527%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21122.96%2182.42%21%21%217.31%214.90%21%402101584917412397443164088ed614%2112000039339269376%21sea%21BD%210%21ABX&curPageLogUid=B1VLp91KJtdF&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006919530273.html?spm=a2g0o.productlist.main.45.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-22&pdp_ext_f=%7B%22order%22%3A%222955%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21116.34%2170.29%21%21%210.96%210.58%21%40212e509017412396850451535e9268%2112000038727156421%21sea%21BD%210%21ABX&curPageLogUid=ZVilIz9DQoEv&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005604632738.html?spm=a2g0o.productlist.main.33.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-16&pdp_ext_f=%7B%22order%22%3A%221886%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21690.97%21345.48%21%21%2141.08%2120.54%21%40212e509017412396850451535e9268%2112000033721485692%21sea%21BD%210%21ABX&curPageLogUid=CzHai6f5tJdb&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007062033530.html?spm=a2g0o.productlist.main.27.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-13&pdp_ext_f=%7B%22order%22%3A%221784%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21184.18%21123.46%21%21%2110.95%217.34%21%40212e509017412396850451535e9268%2112000039277918624%21sea%21BD%210%21ABX&curPageLogUid=WVVTt151orpp&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007082496782.html?spm=a2g0o.productlist.main.25.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-12&pdp_ext_f=%7B%22order%22%3A%221370%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21256.51%21171.90%21%21%2115.25%2110.22%21%40212e509017412396850451535e9268%2112000039339177204%21sea%21BD%210%21ABX&curPageLogUid=kkv4sdvoUtFb&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005097253008.html?spm=a2g0o.productlist.main.19.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-9&pdp_ext_f=%7B%22order%22%3A%2218548%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21116.34%2170.29%21%21%210.96%210.58%21%40212e509017412396850451535e9268%2112000031646743549%21sea%21BD%210%21ABX&curPageLogUid=aIPWlhE9kAYe&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005096978578.html?spm=a2g0o.productlist.main.17.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-8&pdp_ext_f=%7B%22order%22%3A%221403%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21166.02%21107.85%21%21%211.37%210.89%21%40212e509017412396850451535e9268%2112000031645636635%21sea%21BD%210%21ABX&curPageLogUid=QrnMTtdAlCCN&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007980888540.html?spm=a2g0o.productlist.main.9.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-4&pdp_ext_f=%7B%22order%22%3A%223135%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%2199.37%2169.07%21%21%210.82%210.57%21%40212e509017412396850451535e9268%2112000043137305085%21sea%21BD%210%21ABX&curPageLogUid=nnsC4iZfjQdy&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005006036782336.html?spm=a2g0o.productlist.main.5.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-2&pdp_ext_f=%7B%22order%22%3A%226081%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21123.29%2181.41%21%21%217.33%214.84%21%40212e509017412396850451535e9268%2112000035427183034%21sea%21BD%210%21ABX&curPageLogUid=VjfAtjvTf18V&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005005284339594.html?spm=a2g0o.productlist.main.3.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-1&pdp_ext_f=%7B%22order%22%3A%223356%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21126.03%2182.40%21%21%211.04%210.68%21%40212e509017412396850451535e9268%2112000032487576229%21sea%21BD%210%21ABX&curPageLogUid=GBscXLFSqiNt&utparam-url=scene%3Asearch%7Cquery_from%3A",
"https://www.aliexpress.com/item/1005007419321372.html?spm=a2g0o.productlist.main.1.705a6166mYNOyn&algo_pvid=1c2f094c-01e3-4824-b684-29502826512d&algo_exp_id=1c2f094c-01e3-4824-b684-29502826512d-0&pdp_ext_f=%7B%22order%22%3A%221452%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21372.90%21111.85%21%21%2122.17%216.65%21%40212e509017412396850451535e9268%2112000040673563506%21sea%21BD%210%21ABX&curPageLogUid=LvVF2NMdZPyS&utparam-url=scene%3Asearch%7Cquery_from%3A"
]
print(len(url3))

31


In [ ]:
max_feedback3=["1427","585","829","523","374","260","320","461","311","142","1026","243","127","421","197","492","309","1207","3282","818","389","1167","252","595","484","6044","509","657","1769","647","329"
]
print(len(max_feedback3))

31


In [ ]:
# Run the scraper for all URLs and save to a single CSV
scrape_multiple_urls(url3, max_feedback3)

🔗 URL 1/31: https://www.aliexpress.com/item/1005006471222041.html?spm=a2g0o.productlist.main.115.4bbb1611UQA8lu&algo_pvid=0312d218-e564-4d6a-875b-ed9162f5814c&algo_exp_id=0312d218-e564-4d6a-875b-ed9162f5814c-57&pdp_ext_f=%7B%22order%22%3A%227416%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21833.74%21458.07%21%21%216.88%213.78%21%40210141f717412384359201548eaffc%2112000037317581168%21sea%21BD%210%21ABX&curPageLogUid=BY8QaYj8G0m5&utparam-url=scene%3Asearch%7Cquery_from%3A
📝 Number of feedbacks collected from this URL: 1080

🔗 URL 2/31: https://www.aliexpress.com/item/1005002908009280.html?spm=a2g0o.productlist.main.41.4bbb1611UQA8lu&algo_pvid=97c93f81-7324-45d8-ac55-5c01c670525a&algo_exp_id=97c93f81-7324-45d8-ac55-5c01c670525a-20&pdp_ext_f=%7B%22order%22%3A%223127%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21BDT%21325.98%21208.43%21%21%212.69%211.72%21%402101062a17412384987491595ec517%2112000022722852421%21sea%21BD%210%21ABX&curPageLogUid=5MyXzCpqzDx9&utparam-url=scene%3Asearch%

ApifyApiError: Monthly usage hard limit exceeded

# **Merge 3 phase scraped review and remove empty reviews**

In [ ]:
import pandas as pd

# Function to concatenate two CSVs and clean the data
def concatenate_and_clean_csv(csv1, csv2,csv3, output_csv):
    # Read the two CSV files into DataFrames
    df1 = pd.read_csv(csv1)
    df2 = pd.read_csv(csv2)
    df3 = pd.read_csv(csv3)

    # Concatenate the two DataFrames
    combined_df = pd.concat([df1, df2,df3], ignore_index=True)

    # Drop rows where the 'reviewContent' column is empty or NaN
    cleaned_df = combined_df.dropna(subset=['reviewContent'])

    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_csv, index=False)

    print(f"✅ The cleaned data has been saved to {output_csv}")
    return cleaned_df

# Example usage:
csv_file1 = "/content/finalcleaned_user_feedbacks2.csv"  # Path to the first CSV file
csv_file2 = "/content/Final_all_user_feedbacks22.csv"  # Path to the second CSV file
csv_file3 = "/content/Final_all_user_feedbacks221.csv"  # Path to the second CSV file

output_csv = "finalcleaned_user_feedbacks3.csv"  # Path to save the cleaned CSV

# Call the function to concatenate and clean
cleaned_data = concatenate_and_clean_csv(csv_file1, csv_file2,csv_file3, output_csv)

# Optionally, print the first few rows of the cleaned data
print(cleaned_data.head())


✅ The cleaned data has been saved to finalcleaned_user_feedbacks3.csv
                                          productURL        userName  \
0  https://www.aliexpress.com/item/10050068766578...           A***n   
1  https://www.aliexpress.com/item/10050068766578...  AliExpress 쇼핑객   
2  https://www.aliexpress.com/item/10050068766578...  AliExpress 쇼핑객   
3  https://www.aliexpress.com/item/10050068766578...           V***r   
4  https://www.aliexpress.com/item/10050068766578...           B***s   

  userCountry  userStar                                      reviewContent  \
0          US       5.0  좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋은 좋...   
1          US       5.0              아직 시도하지 않았지만 내일 내 차를 청소할 것입니다. 감사합니다.   
2          US       5.0              광고가 말하는 것처럼 정확히. 강한 흡입 경량은 놀라운 일을합니다.   
3          US       5.0                                        인상적인 작은 백신.   
4          US       5.0                                           잘 작동합니다.   

          reviewTime  
0  06

# **Preprocessing the Datset for Take Only English Reviews**

In [ ]:
import pandas as pd
import re
import string
from langdetect import detect

# Load your dataset
df = pd.read_csv('/content/finalcleaned_user_feedbacks3.csv')
print("First:", df.shape)

# Define a function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

print("Detecting languages...")
# Detect the language of each review
df['language'] = df['reviewContent'].apply(detect_language)

# Keep only the reviews in English
df = df[df['language'] == 'en']

# Save the processed data
df.to_csv("/content/processed_user_reviewContent.csv", index=False)

print("Processed DataFrame size (rows, columns):", df.shape)
print(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


First: (22915, 6)
Detecting languages...
Processed DataFrame size (rows, columns): (17585, 7)
                                           productURL            userName  \
90  https://www.aliexpress.com/item/10050068766578...  AliExpress Shopper   
91  https://www.aliexpress.com/item/10050068766578...               A***r   
92  https://www.aliexpress.com/item/10050068766578...               J***s   
93  https://www.aliexpress.com/item/10050068766578...               Б***ч   
94  https://www.aliexpress.com/item/10050068766578...               A***r   

   userCountry  userStar                                      reviewContent  \
90          UA       5.0                                small wont work car   
91          BR       5.0  satisfied product really quite strong bad poin...   
92         NaN       5.0                            satisfied little vacuum   
93          RU       5.0                           bomb work like hurricane   
94          KR       5.0                        